# CatBoost Ranker - Ensemble of All Models

## 1. Setup and Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
os.chdir(project_root)

import cornac
import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import yaml
import pickle
import torch
from tqdm.auto import tqdm

from catboost import CatBoostRanker, Pool

from rectools import Columns
from rectools.dataset import Dataset
from tecd_retail_recsys.data.bert4rec_dataset import BERT4RecDatasetBuilder
from rectools.models import BERT4RecModel, ImplicitALSWrapperModel, LightFMWrapperModel

from recommenders.models.cornac.cornac_utils import predict_ranking
from recommenders.utils.constants import SEED

from tecd_retail_recsys.data import DataPreprocessor
from tecd_retail_recsys.metrics import calculate_metrics
from tecd_retail_recsys.models import TopPopular, TopPersonal, EASE, iALS, TIFUKNN, SASRec

import warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

print(f"System version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")

System version: 3.11.14 (main, Oct 28 2025, 12:11:54) [Clang 20.1.4 ]
PyTorch version: 2.10.0


In [3]:
pwd

'/Users/dlyapin/Documents/git_projects/hse-masters-thesis-2026'

In [26]:
TOP_K = 100
RANDOM_SEED = 42

BERT4REC_MODEL_PATH = './models_test/bert4rec_model.pkl'
BIVAE_MODEL_PATH = './models_test/bivae_model.pkl/BiVAECF/2026-02-27_16-55-09-286650.pkl'
BPR_MODEL_PATH = './models_test/bpr_model.pkl/BPR/2026-02-27_19-35-26-286646.pkl'
LIGHTFM_MODEL_PATH = './models_test/lightfm_model.pkl'
SASREC_CHECKPOINT_PATH = './models_test/sasrec_test_best_state.pth'

FEATURES_PATH = 'models_test/data/all_recs_features_test.parquet'

CATBOOST_MODEL_PATH = './models_test/catboost_ranker_model.cbm'

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

## 2. Load and Prepare Data

In [6]:
dp = DataPreprocessor(
    day_begin=1082, 
    day_end=1308, 
    val_days=20, 
    test_days=20, 
    min_user_interactions=1, 
    min_item_interactions=20
)
train_df, val_df, test_df = dp.preprocess()

print(f"Train shape: {train_df.shape}")
print(f"Val shape: {val_df.shape}")
print(f"Test shape: {test_df.shape}")

user_id_map = dp.user_to_idx
item_id_map = dp.item_to_idx
inv_user_id_map = {v: k for k, v in user_id_map.items()}
inv_item_id_map = {v: k for k, v in item_id_map.items()}

n_users = train_df['user_id'].nunique()
n_items = train_df['item_id'].nunique()

print(f"\nNumber of users: {n_users}")
print(f"Number of items: {n_items}")

Starting data preprocessing...
Loading events from t_ecd_small_partial/dataset/small/retail/events
Loaded 236,479,226 total events
Loading items data from t_ecd_small_partial/dataset/small/retail/items.pq
Loaded 250,171 items with features: ['item_id', 'item_brand_id', 'item_category', 'item_subcategory', 'item_price', 'item_embedding']
Merged item features. Data shape: (236479226, 12)
Filtered to 3,758,762 events with action_type='added-to-cart'
After filtering (min_user_interactions=1, min_item_interactions=20): 3,249,972 events, 84,944 users, 30,954 items
Created mappings: 84944 users, 30954 items
Temporal split - Train: days < 1269 (902,543 events), Val: days 1269-1288 (228,339 events), Test: days >= 1289 (223,395 events)
Users in each part (train, val, test) - 7425
Train shape: (902543, 12)
Val shape: (228339, 12)
Test shape: (223395, 12)

Number of users: 7425
Number of items: 30751


In [ ]:
train_orig = train_df.copy()
val_orig = val_df.copy()
trainval_df = pd.concat([train_orig, val_orig], ignore_index=True)

joined = dp.get_grouped_data(train_orig, val_orig, test_df)
joined['train_val_interactions'] = joined['train_interactions'] + joined['val_interactions']

test_users = test_df['user_id'].unique()

print(f"Evaluation users: {len(test_users)}")
print(f"Joined data shape: {joined.shape}")

Evaluation users: 7425
Joined data shape: (7425, 5)


## 3. Load Baseline Models and Generate Recommendations

### 3.1 TopPopular

In [9]:
print("Training TopPopular...")
toppopular = TopPopular()
toppopular.fit(joined, col='train_val_interactions')

toppopular_recs_grouped = joined.copy()
toppopular_recs_grouped['toppopular_recs'] = toppopular.predict(joined, topn=TOP_K, return_scores=False)
toppopular_recs_grouped = toppopular_recs_grouped[['user_id', 'toppopular_recs']]

print(f"TopPopular recommendations shape: {toppopular_recs_grouped.shape}")

Training TopPopular...
TopPopular recommendations shape: (7425, 2)


### 3.2 TopPersonal

In [10]:
print("Training TopPersonal...")
toppersonal = TopPersonal()
toppersonal.fit(joined, col='train_val_interactions')

toppersonal_recs_grouped = joined.copy()
toppersonal_recs_grouped['toppersonal_recs'] = toppersonal.predict(joined, topn=TOP_K, return_scores=False)
toppersonal_recs_grouped = toppersonal_recs_grouped[['user_id', 'toppersonal_recs']]

print(f"TopPersonal recommendations shape: {toppersonal_recs_grouped.shape}")

Training TopPersonal...
TopPersonal recommendations shape: (7425, 2)


### 3.3 iALS

In [11]:
print("Training iALS...")
matrix_train, idx_to_item_train = dp.get_interactions_matrix(joined, col='train_val_interactions')
ials_recs_grouped = joined.copy()

ials = iALS()


ials = iALS(idx_to_item=idx_to_item_train)
ials.fit(matrix_train)    
ials_recs_grouped['ials_recs'] = ials.predict(joined, topn=100)

ials_recs_grouped = ials_recs_grouped[['user_id', 'ials_recs']]

print(f"iALS recommendations shape: {ials_recs_grouped.shape}")

Training iALS...
Iter № 1/10:


Updating items: 100%|██████████| 30923/30923 [00:15<00:00, 1946.94it/s]


Iter № 2/10:


Updating items: 100%|██████████| 30923/30923 [00:14<00:00, 2127.05it/s]


Iter № 3/10:


...

Iter № 9/10:


Updating items: 100%|██████████| 30923/30923 [00:20<00:00, 1533.28it/s]


Iter № 10/10:


Updating items: 100%|██████████| 30923/30923 [00:18<00:00, 1678.66it/s]


iALS recommendations shape: (7425, 2)


### 3.4 TIFUKNN

In [12]:
print("Training TIFUKNN...")
tifu_recs_grouped = joined.copy()
tifu = TIFUKNN(n_neighbors=1000)
tifu.fit(joined, col='train_val_interactions')
tifu_recs_grouped['tifuknn_recs'] = tifu.predict(joined, topn=100)
tifu_recs_grouped = tifu_recs_grouped[['user_id', 'tifuknn_recs']]

print(f"TIFUKNN recommendations shape: {tifu_recs_grouped.shape}")


Training TIFUKNN...
Building TIFU-KNN PIF vectors for 7425 users and 30923 items...


Computing PIF: 100%|██████████| 7425/7425 [00:01<00:00, 5193.12it/s]


Precomputing user similarities and neighbors...
Finding 1000 nearest neighbors for each user...


Finding neighbors: 100%|██████████| 7425/7425 [00:00<00:00, 10005.61it/s]


Precomputing collaborative signals...


...

TIFU-KNN training completed!
TIFU-KNN training completed!
Generating recommendations for 7425 users...
Masking interacted items...


Creating masks: 100%|██████████| 7425/7425 [00:00<00:00, 38223.73it/s]


Computing top-N recommendations...


Extracting top-N: 100%|██████████| 7425/7425 [00:01<00:00, 4321.24it/s]


TIFUKNN recommendations shape: (7425, 2)


## 4. Load Advanced Models and Generate Recommendations

### 4.1 BERT4Rec

In [15]:
builder = BERT4RecDatasetBuilder(trainval_df)
dataset, item_net_config = builder.build_dataset(
    use_item_embeddings=False,
    use_price_features=True,
    use_temporal_features=False,
    use_user_features=False,
)


🏗️  BERT4Rec Dataset Builder
✅ Interactions: 1130882 строк
📦 Добавление базовых item features...
  ✅ Brand: 30923 items
  ✅ Category: 30367 items
  ✅ Subcategory: 30367 items
💰 Добавление price features...
  ✅ Price buckets: 30923 items, 10 categories
  ✅ Price tier: 30923 items
  ✅ Price in category: 29320 items

📦 Итого item features: 182823 строк
   Фичи: ['brand', 'category', 'subcategory', 'price_bucket', 'price_tier', 'price_in_category']
   Уникальных товаров: 30923

🔨 Создание RecTools Dataset...
✅ Dataset: 7425 users, 30923 items

✅ ItemNet: ID + Categorical
✅ Dataset готов к использованию!



In [21]:

print(f"Loading BERT4Rec from {BERT4REC_MODEL_PATH}...")

def get_cpu_trainer(**kwargs):
    from pytorch_lightning import Trainer
    trainer_kwargs = {
        'accelerator': 'cpu',
        'devices': 1,
        'max_epochs': 100,
    }
    trainer_kwargs.update(kwargs)
    return Trainer(**trainer_kwargs)


bert4rec_model = BERT4RecModel.load(BERT4REC_MODEL_PATH)

bert4rec_recs = bert4rec_model.recommend(
    users=test_users,
    dataset=dataset,
    k=TOP_K,
    filter_viewed=False,
    on_unsupported_targets="ignore"
)

bert4rec_recs_grouped = bert4rec_recs.groupby('user_id', as_index=False)['item_id'].agg(list)
bert4rec_recs_grouped.columns = ['user_id', 'bert4rec_recs']

print(f"BERT4Rec recommendations shape: {bert4rec_recs_grouped.shape}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Loading BERT4Rec from ./models_test/bert4rec_model.pkl...


Restoring states from the checkpoint path at /var/folders/5t/b05_gxx17hnftz_n3c82pt4h0000gn/T/tmp5m4o6kkh

  | Name        | Type                     | Params | Mode 
-----------------------------------------------------------------
0 | torch_model | TransformerTorchBackbone | 9.5 M  | train
-----------------------------------------------------------------
9.5 M     Trainable params
0         Non-trainable params
9.5 M     Total params
37.985    Total estimated model params size (MB)
40        Modules in train mode
0         Modules in eval mode
Restored all states from the checkpoint at /var/folders/5t/b05_gxx17hnftz_n3c82pt4h0000gn/T/tmp5m4o6kkh
`Trainer.fit` stopped: No training batches.


BERT4Rec recommendations shape: (7425, 2)


### 4.2 BiVAE

In [24]:
print(f"Loading BiVAE from {BIVAE_MODEL_PATH}...")

trainval_agg = trainval_df.rename(columns={'user_id': 'userID', 'item_id': 'itemID'}).groupby(['userID', 'itemID']).size().reset_index(name='count')
trainval_data = [(str(row['userID']), str(row['itemID']), float(row['count'])) 
              for _, row in trainval_agg.iterrows()]
trainval_set = cornac.data.Dataset.from_uir(trainval_data, seed=SEED)


bivae_model = cornac.models.BiVAECF.load(BIVAE_MODEL_PATH)
bivae_model.train_set = trainval_set

bivae_recs = predict_ranking(bivae_model, trainval_set, usercol='userID', itemcol='itemID', remove_seen=False)
bivae_recs_top100 = (bivae_recs
          .sort_values(['userID', 'prediction'], ascending=[True, False])
          .groupby('userID', as_index=False)
          .head(100)
         )
bivae_recs_top100['userID'] = bivae_recs_top100['userID'].astype(np.int64)
bivae_recs_top100['itemID'] = bivae_recs_top100['itemID'].astype(np.int64)

bivae_recs_grouped = bivae_recs_top100.groupby('userID', as_index=False).itemID.agg(list)
bivae_recs_grouped.columns = ['user_id', 'bivae_recs']

print(f"BiVAE recommendations shape: {bivae_recs_grouped.shape}")

Loading BiVAE from ./models_test/bivae_model.pkl/BiVAECF/2026-02-27_16-55-09-286650.pkl...
BiVAE recommendations shape: (7425, 2)


### 4.3 BPR

In [25]:
print(f"Loading BPR from {BPR_MODEL_PATH}...")

bpr_model = cornac.models.BPR.load(BPR_MODEL_PATH)
bpr_model.train_set = trainval_set

bpr_recs = predict_ranking(bpr_model, trainval_set, usercol='userID', itemcol='itemID', remove_seen=False)
bpr_recs_top100 = (bpr_recs
          .sort_values(['userID', 'prediction'], ascending=[True, False])
          .groupby('userID', as_index=False)
          .head(100)
         )
bpr_recs_top100['userID'] = bpr_recs_top100['userID'].astype(np.int64)
bpr_recs_top100['itemID'] = bpr_recs_top100['itemID'].astype(np.int64)

bpr_recs_grouped = bpr_recs_top100.groupby('userID', as_index=False).itemID.agg(list)
bpr_recs_grouped.columns = ['user_id', 'bpr_recs']

print(f"BPR recommendations shape: {bpr_recs_grouped.shape}")

Loading BPR from ./models_test/bpr_model.pkl/BPR/2026-02-27_19-35-26-286646.pkl...
BPR recommendations shape: (7425, 2)


### 4.4 LightFM

In [28]:
builder = BERT4RecDatasetBuilder(trainval_df)
dataset, item_net_config = builder.build_dataset(
    use_item_embeddings=False,
    use_price_features=False,
    use_temporal_features=False,
    use_user_features=True,
    n_factors=512
)


🏗️  BERT4Rec Dataset Builder
✅ Interactions: 1130882 строк
📦 Добавление базовых item features...
  ✅ Brand: 30923 items
  ✅ Category: 30367 items
  ✅ Subcategory: 30367 items
👤 Добавление user features...
  ✅ Interaction level: 7425 users
  ✅ User price segment: 7425 users
  ✅ Favorite brand: 7425 users
  ✅ Favorite category: 7425 users
  ✅ User diversity: 7425 users
  ✅ User recency: 7425 users

📦 Итого item features: 91657 строк
   Фичи: ['brand', 'category', 'subcategory']
   Уникальных товаров: 30923

👤 Итого user features: 44550 строк
   Фичи: ['interaction_level', 'user_price_segment', 'favorite_brand', 'favorite_category', 'user_diversity', 'user_recency']
   Уникальных пользователей: 7425

🔨 Создание RecTools Dataset...
✅ Dataset: 7425 users, 30923 items

✅ ItemNet: ID + Categorical
✅ Dataset готов к использованию!



In [29]:
print(f"Loading LightFM from {LIGHTFM_MODEL_PATH}...")


lightfm_model = LightFMWrapperModel.load(LIGHTFM_MODEL_PATH)

lightfm_recs = lightfm_model.recommend(
    users=test_users,
    dataset=dataset,
    k=TOP_K,
    filter_viewed=False,
    on_unsupported_targets="ignore"
)

lightfm_recs_grouped = lightfm_recs.groupby('user_id', as_index=False)['item_id'].agg(list)
lightfm_recs_grouped.columns = ['user_id', 'lightfm_recs']

print(f"LightFM recommendations shape: {lightfm_recs_grouped.shape}")

Loading LightFM from ./models_test/lightfm_model.pkl...
LightFM recommendations shape: (7425, 2)


### 4.5 SASRec

In [31]:
print(f"Loading SASRec from {SASREC_CHECKPOINT_PATH}...")

sasrec_model = SASRec(
    checkpoint_path=SASREC_CHECKPOINT_PATH,
    max_seq_len=100,
    device='cpu',
    batch_size=128
)

sasrec_model.load_model()

sasrec_recs = sasrec_model.predict(joined[['user_id', 'train_val_interactions', 'test_interactions']].rename(columns={'train_val_interactions': 'train_interactions', 'test_interactions': 'val_interactions'}), topn=100, return_scores=False)
sasrec_recs_grouped = joined.copy()
sasrec_recs_grouped['sasrec_recs'] = sasrec_recs
sasrec_recs_grouped = sasrec_recs_grouped[['user_id', 'sasrec_recs']]

print(f"SASRec recommendations shape: {sasrec_recs_grouped.shape}")

Loading SASRec from ./models_test/sasrec_test_best_state.pth...
SASRec recommendations shape: (7425, 2)


## 5. Merge All Recommendations

In [32]:
# Merge all recommendations
all_recs = joined.copy()

for recs_df, col_name in [
    (toppopular_recs_grouped, 'toppopular_recs'),
    (toppersonal_recs_grouped, 'toppersonal_recs'),
    (ials_recs_grouped, 'ials_recs'),
    (tifu_recs_grouped, 'tifuknn_recs'),
    (bert4rec_recs_grouped, 'bert4rec_recs'),
    (bivae_recs_grouped, 'bivae_recs'),
    (bpr_recs_grouped, 'bpr_recs'),
    (lightfm_recs_grouped, 'lightfm_recs'),
    (sasrec_recs_grouped, 'sasrec_recs')
]:
    all_recs = all_recs.merge(recs_df, on='user_id', how='left')
    all_recs[col_name] = all_recs[col_name].apply(
        lambda x: x if isinstance(x, list) else []
    )

print(f"All recommendations merged. Shape: {all_recs.shape}")
print(f"Columns: {all_recs.columns.tolist()}")

All recommendations merged. Shape: (7425, 14)
Columns: ['user_id', 'train_interactions', 'val_interactions', 'test_interactions', 'train_val_interactions', 'toppopular_recs', 'toppersonal_recs', 'ials_recs', 'tifuknn_recs', 'bert4rec_recs', 'bivae_recs', 'bpr_recs', 'lightfm_recs', 'sasrec_recs']


In [33]:
all_recs.to_parquet("models_test/data/all_recs_test.parquet", index=False)

## 6. Create Combined Candidate Pool

In [34]:
def combine_candidates(row):
    """Combine all model recommendations for a user, removing duplicates."""
    candidates = set()
    
    for col in ['toppopular_recs', 'toppersonal_recs', 
                'ials_recs',  'tifuknn_recs', 
                'bert4rec_recs', 'bivae_recs', 'bpr_recs', 
                'lightfm_recs', 'sasrec_recs'
                ]:
        if isinstance(row[col], list):
            candidates.update(row[col])
    
    return list(candidates)

all_recs['combined_candidates'] = all_recs.apply(combine_candidates, axis=1)
all_recs['num_candidates'] = all_recs['combined_candidates'].apply(len)

print(f"Average number of unique candidates per user: {all_recs['num_candidates'].mean():.1f}")
print(f"Min candidates: {all_recs['num_candidates'].min()}")
print(f"Max candidates: {all_recs['num_candidates'].max()}")

Average number of unique candidates per user: 489.8
Min candidates: 350
Max candidates: 689


## 7. Load Features and Create Training Data

In [35]:
print("Creating training data with features...")

training_pairs = []

for idx, row in tqdm(all_recs.iterrows(), total=len(all_recs), desc="Creating pairs"):
    user_id = row['user_id']
    candidates = row['combined_candidates']
    val_items = [item_id for item_id, _, _ in row['train_interactions']]
    
    for item_id in candidates:
        target = 1 if item_id in val_items else 0
        training_pairs.append({
            'user_id': user_id,
            'item_id': item_id,
            'target': target
        })

training_df_pd = pd.DataFrame(training_pairs)
print(f"\nTraining pairs created: {len(training_df_pd)}")
print(f"Positive samples: {(training_df_pd['target'] == 1).sum()}")
print(f"Negative samples: {(training_df_pd['target'] == 0).sum()}")
print(f"Positive rate: {(training_df_pd['target'] == 1).mean():.4f}")

Creating training data with features...


Creating pairs:   0%|          | 0/7425 [00:00<?, ?it/s]


Training pairs created: 3636665
Positive samples: 548534
Negative samples: 3088131
Positive rate: 0.1508


In [36]:
training_df_pd.to_parquet("models_test/data/training_df_pd_test.parquet", index=False)

In [37]:
# collect features with pairs (user_id, item_id)

from tecd_retail_recsys.data.features import add_features_to_samples

features = add_features_to_samples(samples_df=training_df_pd, train_df=trainval_df, dp=dp, end_day=1288)
features.drop(columns=['item_brand_embedding', 'item_embedding', 'brand_embedding'], inplace=True)
features.to_parquet(FEATURES_PATH, index=False)
del features

user_features: (7425, 14), nans: 0
item_features: (30954, 19), nans: 0
brand_features: (5580, 6), nans: 0
user_item_features: (782025, 9), nans: 0
user_brand_features: (37811, 12), nans: 0
user_category_features: (31641, 12), nans: 0
done! nans:  4365724


In [40]:
# go to polars
features_df = pl.read_parquet(FEATURES_PATH)

In [41]:
print("Adding candidate generator ranks as features...")

all_recs = pd.read_parquet("models_test/data/all_recs_test.parquet")

model_columns = [
    'toppopular_recs', 'toppersonal_recs', 'ials_recs', 
    'tifuknn_recs', 'bert4rec_recs', 'bivae_recs', 'bpr_recs', 
    'lightfm_recs', 'sasrec_recs'
]

for col in model_columns:
    if col in all_recs.columns:
        if all_recs[col].dtype == 'object':
            all_recs[col] = all_recs[col].apply(
                lambda x: ast.literal_eval(x) if isinstance(x, str) else x
            )


rank_features = {}

for model_col in model_columns:
    print(f"Processing {model_col}...")
    model_name = model_col.replace('_recs', '')
    
    user_item_to_rank = {}
    
    for idx, row in all_recs.iterrows():
        user_id = row['user_id']
        recs = row[model_col].tolist()
        
        if isinstance(recs, list) and len(recs) > 0:
            for rank, item_id in enumerate(recs, start=1):
                user_item_to_rank[(user_id, item_id)] = rank
    
    rank_features[model_name] = user_item_to_rank
    print(f"  Created {len(user_item_to_rank)} user-item-rank mappings")


Adding candidate generator ranks as features...
Processing toppopular_recs...
  Created 742500 user-item-rank mappings
Processing toppersonal_recs...
  Created 523920 user-item-rank mappings
Processing ials_recs...
  Created 742500 user-item-rank mappings
Processing tifuknn_recs...
  Created 742500 user-item-rank mappings
Processing bert4rec_recs...
  Created 742500 user-item-rank mappings
Processing bivae_recs...
  Created 742500 user-item-rank mappings
Processing bpr_recs...
  Created 742500 user-item-rank mappings
Processing lightfm_recs...
  Created 742500 user-item-rank mappings
Processing sasrec_recs...
  Created 742500 user-item-rank mappings


In [42]:
training_df_pl = pl.from_pandas(training_df_pd)

training_data = training_df_pl.join(
    features_df,
    on=['user_id', 'item_id'],
    how='left'
)

print(f"Training data after joining features: {training_data.shape}")
print(f"Missing features: {training_data.null_count().sum()[0]} nulls")

Training data after joining features: (3636665, 64)
Missing features: shape: (1, 64)
┌─────────┬─────────┬────────┬─────────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ user_id ┆ item_id ┆ target ┆ item_brand_ ┆ … ┆ user_categ ┆ user_categ ┆ user_categ ┆ user_categ │
│ ---     ┆ ---     ┆ ---    ┆ id          ┆   ┆ ory_days_s ┆ ory_days_s ┆ ory_lifeti ┆ ory_orders │
│ u32     ┆ u32     ┆ u32    ┆ ---         ┆   ┆ ince_first ┆ ince_last_ ┆ me         ┆ _share     │
│         ┆         ┆        ┆ u32         ┆   ┆ …          ┆ …          ┆ ---        ┆ ---        │
│         ┆         ┆        ┆             ┆   ┆ ---        ┆ ---        ┆ u32        ┆ u32        │
│         ┆         ┆        ┆             ┆   ┆ u32        ┆ u32        ┆            ┆            │
╞═════════╪═════════╪════════╪═════════════╪═══╪════════════╪════════════╪════════════╪════════════╡
│ 0       ┆ 0       ┆ 0      ┆ 0           ┆ … ┆ 0          ┆ 0          ┆ 95524      ┆ 95524      │
└─────

In [43]:
numeric_cols = [col for col in training_data.columns if training_data[col].dtype in [pl.Float64, pl.Float32, pl.Int64, pl.Int32]]
print(f"Numeric columns to fill: {len(numeric_cols)}")

medians = {}
for col in numeric_cols:
    if col not in ['user_id', 'item_id', 'target']:
        median_val = training_data[col].median()
        medians[col] = median_val if median_val is not None else 0.0

print(f"Computed medians for {len(medians)} columns")

fill_expressions = [
    pl.col(col).fill_null(medians[col]) 
    for col in medians.keys()
]

if fill_expressions:
    training_data = training_data.with_columns(fill_expressions)

print(f"Filled numeric columns with median values")

categorical_cols = [col for col in training_data.columns if training_data[col].dtype == pl.Utf8]
print(f"Categorical columns to fill: {len(categorical_cols)}")

if categorical_cols:
    training_data = training_data.with_columns([
        pl.col(col).fill_null('unknown') for col in categorical_cols
    ])

print(f"After filling nulls: {training_data.null_count().to_numpy().sum()} nulls remaining")

Numeric columns to fill: 55
Computed medians for 52 columns
Filled numeric columns with median values
Categorical columns to fill: 5
After filling nulls: 0 nulls remaining


## 8. Prepare Features for CatBoost

In [50]:
categorical_features = [
    'user_id', 'item_id', 'item_brand_id', 'user_os', 'user_main_subdomain',
    'user_socdem_cluster', 'user_region', 'item_category', 'item_subcategory',
]

categorical_features = [col for col in categorical_features if col in training_data.columns]
print(f"Categorical features: {categorical_features}")

training_data_pd = training_data.to_pandas()

for model_name, user_item_rank_map in rank_features.items():
    rank_col_name = f'rank_{model_name}'
    
    ranks = []
    for idx, row in training_data_pd.iterrows():
        user_id = row['user_id']
        item_id = row['item_id']
        
        rank = user_item_rank_map.get((user_id, item_id), 101)
        ranks.append(rank)
    
    training_data_pd[rank_col_name] = ranks
    print(f"  Added {rank_col_name}: min={min(ranks)}, max={max(ranks)}, mean={np.mean(ranks):.2f}")

X = training_data_pd.drop('target', axis=1)
y = training_data_pd['target']
groups = training_data_pd['user_id']

for col in categorical_features:
    X[col] = X[col].astype(str)
    
print(f"\nFinal training data:")
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"Number of unique users (groups): {groups.nunique()}")

Categorical features: ['user_id', 'item_id', 'item_brand_id', 'user_os', 'user_main_subdomain', 'user_socdem_cluster', 'user_region', 'item_category', 'item_subcategory']
  Added rank_toppopular: min=1, max=101, mean=90.69
  Added rank_toppersonal: min=1, max=101, mean=92.63
  Added rank_ials: min=1, max=101, mean=90.69
  Added rank_tifuknn: min=1, max=101, mean=90.69
  Added rank_bert4rec: min=1, max=101, mean=90.69
  Added rank_bivae: min=1, max=101, mean=90.69
  Added rank_bpr: min=1, max=101, mean=90.69
  Added rank_lightfm: min=1, max=101, mean=90.69
  Added rank_sasrec: min=1, max=101, mean=90.69

Final training data:
X shape: (3636665, 72)
y shape: (3636665,)
Number of unique users (groups): 7425


## 9. Train CatBoost Ranker

In [51]:
need_to_drop_cols = [
    'user_item_added_to_cart_total',
    'user_item_days_since_first_purchase',
    'user_item_days_since_last_purchase',
    'user_item_top_subdomain',
    'user_item_viewed_total',
    # 'rank_toppersonal'
    ]

X = X.drop(columns=need_to_drop_cols)

In [52]:
# extra feature engineering

ranks_cols = X.columns[X.columns.str.startswith('rank_')].tolist()

X['rank_sum'] = X[ranks_cols].sum(axis=1)
X['rank_mean'] = X[ranks_cols].mean(axis=1)
X['rank_min'] = X[ranks_cols].min(axis=1)
X['rank_max'] = X[ranks_cols].max(axis=1)
X['rank_std'] = X[ranks_cols].std(axis=1)
X['rank_median'] = X[ranks_cols].median(axis=1)

In [53]:
TRAIN_RATIO = 0.9

unique_users = groups.unique()
train_users = np.random.choice(unique_users, size=int(TRAIN_RATIO * len(unique_users)), replace=False)
val_users_cb = np.setdiff1d(unique_users, train_users)

train_mask = groups.isin(train_users)
val_mask = groups.isin(val_users_cb)

X_train, y_train, groups_train = X[train_mask], y[train_mask], groups[train_mask]
X_val, y_val, groups_val = X[val_mask], y[val_mask], groups[val_mask]

print(f"Train set: {X_train.shape[0]} samples, {len(train_users)} users")
print(f"Val set: {X_val.shape[0]} samples, {len(val_users_cb)} users")
print(f"Train positive rate: {y_train.mean():.4f}")
print(f"Val positive rate: {y_val.mean():.4f}")

Train set: 3271800 samples, 6682 users
Val set: 364865 samples, 743 users
Train positive rate: 0.1504
Val positive rate: 0.1544


In [54]:
train_pool = Pool(
    data=X_train,
    label=y_train,
    group_id=groups_train,
    cat_features=categorical_features
)

val_pool = Pool(
    data=X_val,
    label=y_val,
    group_id=groups_val,
    cat_features=categorical_features
)

print("CatBoost pools created successfully")

CatBoost pools created successfully


In [55]:
print("Training CatBoost Ranker...")

model = CatBoostRanker(
    iterations=1000,
    loss_function='YetiRank:top=100',
    custom_metric=['NDCG:top=100', 'PrecisionAt:top=100', 'RecallAt:top=100'],
    eval_metric='NDCG:top=100',
    random_seed=RANDOM_SEED,
    verbose=10,
    early_stopping_rounds=10,
    use_best_model=True
)

model.fit(
    train_pool,
    eval_set=val_pool,
    plot=True
)

print("\nTraining completed!")

Training CatBoost Ranker...


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Groupwise loss function. OneHotMaxSize set to 10
0:	test: 0.9836153	best: 0.9836153 (0)	total: 2.66s	remaining: 44m 18s
10:	test: 0.9839644	best: 0.9841926 (6)	total: 26.3s	remaining: 39m 28s
20:	test: 0.9870245	best: 0.9871223 (17)	total: 48.3s	remaining: 37m 30s
30:	test: 0.9904735	best: 0.9907019 (24)	total: 1m 9s	remaining: 36m 24s
40:	test: 0.9946480	best: 0.9946480 (40)	total: 1m 31s	remaining: 35m 39s
50:	test: 0.9959366	best: 0.9959366 (50)	total: 1m 54s	remaining: 35m 22s
60:	test: 0.9963852	best: 0.9963852 (60)	total: 2m 17s	remaining: 35m 12s
70:	test: 0.9965012	best: 0.9967359 (64)	total: 2m 40s	remaining: 34m 54s
Stopped by overfitting detector  (10 iterations wait)

bestTest = 0.9967359041
bestIteration = 64

Shrink model to first 65 iterations.

Training completed!


In [56]:
pd.DataFrame({'cols': X.columns, 'imp': model.get_feature_importance(type='PredictionValuesChange')}).sort_values('imp', ascending=False).head(20)

,cols,imp
59,rank_toppersonal,56.252169
61,rank_tifuknn,21.877090
10,user_items,7.903729
65,rank_lightfm,6.439335
64,rank_bpr,3.537330
0,user_id,1.123243
60,rank_ials,0.965007
37,user_item_clicked_total,0.824540
4,user_orders,0.405748
36,user_item_lifetime,0.329494


## 10. Evaluate on Validation Set

In [57]:
print("Generating predictions for validation set...")

val_predictions = model.predict(X)
training_data_pd['catboost_score'] = val_predictions

catboost_recs = training_data_pd.sort_values(['user_id', 'catboost_score'], ascending=[True, False])
catboost_recs_grouped = catboost_recs.groupby('user_id').head(100)
catboost_recs_grouped = catboost_recs_grouped.groupby('user_id', as_index=False)['item_id'].agg(list)
catboost_recs_grouped.columns = ['user_id', 'catboost_recs']

print(f"CatBoost recommendations generated for {len(catboost_recs_grouped)} users")

Generating predictions for validation set...
CatBoost recommendations generated for 7425 users


In [58]:
evaluation_df = joined.merge(catboost_recs_grouped, on='user_id', how='left')
evaluation_df['catboost_recs'] = evaluation_df['catboost_recs'].apply(
    lambda x: x if isinstance(x, list) else []
)

print(f"Evaluation dataframe shape: {evaluation_df.shape}")
print(f"Users with CatBoost recommendations: {(evaluation_df['catboost_recs'].str.len() > 0).sum()}")

Evaluation dataframe shape: (7425, 6)
Users with CatBoost recommendations: 7425


In [59]:
metrics_result = calculate_metrics(
    evaluation_df,
    train_col='train_val_interactions',
    gt_col='test_interactions',
    model_preds='catboost_recs',
    verbose=True
)

[Metrics debug] resolved gt_col='test_interactions' item_id_index=0
[Metrics debug] ratings_true shape: (223395, 3) ratings_pred shape: (742500, 3)
  ratings_true dtypes: {'user_id': dtype('int64'), 'item_id': dtype('int64')}
  ratings_pred dtypes: {'user_id': dtype('int64'), 'item_id': dtype('int64')}
  user_id=11 gt_count=9 pred_count=100 overlap=3
  user_id=14 gt_count=56 pred_count=100 overlap=3
  user_id=21 gt_count=43 pred_count=100 overlap=10

At k=10:
  MAP@10       = 0.2665
  NDCG@10      = 0.5517
  Precision@10 = 0.2521
  Recall@10    = 0.0832

At k=100:
  MAP@100       = 0.1125
  NDCG@100      = 0.3202
  Precision@100 = 0.0771
  Recall@100    = 0.2289

Other Metrics:
  MRR                 = 0.3425
  Catalog Coverage    = 0.9953
  Diversity     = 0.9968  [0=same recs for all, 1=unique recs]
  Novelty             = 0.9211
  Serendipity         = 0.0140


## 13. Save Model

In [60]:
model.save_model(CATBOOST_MODEL_PATH)
print(f"Model saved to {CATBOOST_MODEL_PATH}")

Model saved to ./models_test/catboost_ranker_model.cbm


`Наилучший ndcg@100 = 0.3202`